In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from code.mpmath_series import quad_phi
from code.mpmath_series import series_x_mu_b0_pos, bound_series_x_mu_b0_pos, estimate_N_series_x_mu_b0_pos
from code.mpmath_series import series_x_mu_b0_alternating, bound_series_x_mu_b0_alternating, estimate_N_series_x_mu_b0_alternating, bound_series_x_mu_b0_pos_alt

from code.mpmath_series import estimate_N_series_x_mu_b0_pos_fast, estimate_Tk_series_x_mu_b0_pos_fast

In [3]:
from mpmath import *

In [103]:
mp.dps = 50

alpha = mpf('1')
mu = mpf('1/5')
delta = mpf('1/3')
x = mpf('20')

(x - mu) / delta

omega = sqrt((x-mu)**2 + delta**2)
float(alpha * omega), float(((x-mu)**2  / omega * alpha)), float(omega**2)

(19.802805637361367, 19.797194760137916, 392.1511111111111)

### 1. Integral

In [104]:
ref = quad_phi(x, alpha, mu, delta, digits=100)
ref

mpf('0.99999999999498152407898274499113388582081276586782717')

### 2. Series

In [105]:
estimate_N_series_x_mu_b0_pos(x, alpha, mu, delta)

0.9998583212190866 0.003738311137407169 2.2777142458125113e-28 2.4889685032707023e+24 92007.5101535347


mpf('102358.23782867077641452129840896767347004023819298916')

In [106]:
bound_series_x_mu_b0_pos(x, alpha, mu, delta, 324)

-0.5236029192276909

In [107]:
bound_series_x_mu_b0_pos_alt(x, alpha, mu, delta, 324)

1.1843076164254127

In [108]:
float(ref - series_x_mu_b0_pos(x, alpha, mu, delta, 324))

0.42905283138246253

Series

In [63]:
omega = sqrt(delta**2 + (x-mu)**2)
z = (x-mu)**2 * alpha / omega

N = 111

s = mp.zero
for k in range(N):
    r = z**k / fac2(2*k + 1)
    q = besselk(k + 1, alpha * omega)
    s += r * q

remainder = mp.zero
for k in range(N, 151):
    r = z**k / fac2(2*k + 1)
    q = besselk(k + 1, alpha * omega)
    remainder += r * q

print(s, remainder)

0.00012109998016366823172785323900567608180751655325912 0.00059800632145120149253093546653268059276852480002592


Remainder integral representation

In [64]:
M = gamma(N + 3 / 2) * 2 / fac2(2*N + 1) * (2*alpha / omega) / sqrt(pi) * (2*z*alpha / omega)**N
M * quad(lambda t: cos(omega*t) / (t**2 + alpha**2)**(N + 3/2) / (2 - 2*z*alpha / omega / (t**2 + alpha**2)), [0, inf])

mpf('3.3765287901054140702597709310944313996992452304445143')

In [120]:
mp.dps = 100

N = 57

omega = sqrt(delta**2 + (x-mu)**2)
z = (x-mu)**2 * alpha / omega
aw = alpha * omega

s = mp.zero
for k in range(N):
    r = z**k / fac2(2*k + 1)
    q = besselk(k + 1, alpha * omega)
    s += r * q

sN = s

In [121]:
m = 2*z*alpha/omega

S1 = gamma(N + 3 / 2) / fac2(2*N + 1) * (alpha / omega) * sqrt(pi) * 2**(N+1) / sqrt(2*alpha**2*m - m*m) * cosh(sqrt(omega**2 * (2*alpha**2 - m)) / sqrt(2))

S2 = 0
for k in range(N):
    P = alpha**(2*k) * gamma(N + 1 - k) * gamma(k - 1/2) * hyp2f1(1, N + 1 - k, 3/2 - k, 1 - m/(2*alpha**2))
    tk = (-1)**k * omega**(2*k) / factorial(2*k) * P
    S2 += tk
    print(k, tk)

S2 *= (m * 2)**N * alpha**(-2*N -1) / pi / fac2(2*N - 1) * gamma(N + 3 / 2) / fac2(2*N + 1) / omega

S = sN + S1 + S2

SF = S * delta * exp(delta * alpha) / pi  * (x-mu)*alpha / omega  + 1/2

0 -145248709083891547020688815008133181571613603245181952595806706795612740260348.7236151741946133161379
1 -255166177007484443776233929989303845300700984690416050870149813863721116706909.4769744621608377250364
2 69735033509502788927945766358260527227875482804233306232377551594993197963994.33208714465213398082531
3 -25444619019296830032221070055575101163805979635556190372977551026560681012928.74700764032631749749841
4 8282368138864336971533342130736379854896095862089214141399023921963454438378.260165533291054731208044
5 -2387485996178465582300747849449294440683100081228157051349952904249391379392.877951208164410275484748
6 614422863429794699253771649432079289480184700576373449131021067272173454519.33147920035444052603193
7 -142863513561478071631263601661188862896313309242206394613409820182903480206.7894558610647534758122676
8 30359917285134773843590035478323109122838176503224607201127387685883166622.03880612152715470604626033
9 -5957169528910745615366920882925228897705567738949553377806

In [122]:
SF

mpf('0.9999999999949815240789827449911338858208128186971979060299558723623120234495488286091155306303983876099')

In [123]:
ref

mpf('0.9999999999949815240789827449911338858208127658678271720561353594797997531383805084526080460128508754091')

In [124]:
float(ref -SF)

-5.282937073397382e-44

In [125]:
1 - m/(2*alpha**2)

mpf('0.0002833374889498379309563206927034929845637736020128291467833485737891680824846830832167116032096916232208')

Find $N$

minimum value

In [109]:
mp.dps = 20

m = 2*z*alpha/omega

In [110]:
(alpha*omega)**(2*N) * gamma(N - 1/2) / factorial(2*N)

mpf('7.628452673863042012598e+38')

In [111]:
(alpha*omega)**(2*N) * 2**(-2*N - 1/2) * (e / N)**N

mpf('5.5325028818042360357982e+41')

In [112]:
(gamma(N - 1/2) * (alpha * omega)**(2*N) / factorial(2*N) *
 (m * 2)**N * alpha**(-2*N -1) / pi / fac2(2*N - 1) * gamma(N + 3 / 2) / fac2(2*N + 1) / omega)

mpf('8.2471472145458012716278e-51')

In [113]:
(gamma(N - 1/2) * (sqrt(2*m) * omega)**(2*N) / factorial(2*N)
 * alpha**(-1) / pi / fac2(2*N - 1) * gamma(N + 3 / 2) / fac2(2*N + 1) / omega)

mpf('8.2471472145458012719538e-51')

In [114]:
gamma(N - 1/2)  / fac2(2*N - 1) * gamma(N + 3 / 2) / fac2(2*N + 1) / factorial(2*N)

mpf('2.7143607569421291190761e-340')

In [115]:
sqrt(pi) * (e / N) **(2*N) * 2**(-4*N) / 4

mpf('1.9675662293907475303214e-337')

In [116]:
sqrt(pi) * (e / N) **(2*N) * 2**(-4*N - 2) * (m * 2)**N * alpha**(-2*N -1) / pi / omega * (alpha * omega)**(2*N) 

mpf('5.9781325332873755358603e-48')

In [117]:
(sqrt(2 * m) * omega)**(2*N) / alpha / sqrt(pi) / 4 / omega * (e / N) **(2*N) * 2**(-4*N)

mpf('5.9781325332873755361014e-48')

In [118]:
bb = 4 * epsilon * (alpha * omega) * sqrt(pi)
xx = sqrt(2 * m) * omega

In [119]:
-log(bb) / 2 / lambertw(-2*log(bb) / (e * xx))

mpf('56.579095447143668315513')

In [79]:
epsilon = 1e-16
# (alpha * omega / 2)**(2*N) * (e / N)**N = epsilon * sqrt(2)

In [80]:
(alpha * omega / 2)**(2*N) * sqrt(2) * (e / N)**N

mpf('4.7639634057927049330743e+99')

In [81]:
-log(epsilon * sqrt(2)) / lambertw(-log(epsilon * sqrt(2)) / e  / (alpha * omega / 2)**2)

mpf('1721.5481079487894921793')

In [82]:
1 - m/(2*alpha**2)

mpf('0.000044800137626022786670416')